In [2]:
!python3 -m pip install -U git+https://github.com/facebookresearch/audiocraft#egg=audiocraft
# !python3 -m pip install -U audiocraft

  Cloning https://github.com/facebookresearch/audiocraft to /tmp/pip-install-2c2wnt6u/audiocraft_f455c43fcc7c4852b76ca644c7888233
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/audiocraft /tmp/pip-install-2c2wnt6u/audiocraft_f455c43fcc7c4852b76ca644c7888233
  Resolved https://github.com/facebookresearch/audiocraft to commit adf0b04a4452f171970028fcf80f101dd5e26e19
  Preparing metadata (setup.py) ... done


In [12]:
from audiocraft.models import musicgen
from audiocraft.utils.notebook import display_audio
import torch

import numpy as np
import torch
import scipy.io.wavfile as wavfile
from IPython.display import FileLink

/home/csi/music_gen_playground/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
model = musicgen.MusicGen.get_pretrained('facebook/musicgen-small', device='cuda')
model.set_generation_params(duration=45)

/home/csi/music_gen_playground/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [ ]:
# Demo

res = model.generate([
    'A legato, oriental-inspired melody for solo synthesizer, featuring a warm, analog-like timbre with a mix of microtones and Middle Eastern modal inflections.',
],
    progress=True)
display_audio(res, 32000)

# Move the tensor to CPU if it's on GPU and convert to NumPy array
audio_data = res.cpu().numpy()

# Save the audio file
filename = "koto_shamisen_m.wav"
sample_rate = 32000  # Sample rate used in display_audio
wavfile.write(filename, sample_rate, audio_data)

# Provide a download link
display(FileLink(filename))

See our repo https://github.com/facebookresearch/audiocraft for more details on how to use this model!

See also [MusicGen Gradio Demo](https://colab.research.google.com/drive/1-Xe9NCdIs2sCUbiSmwHXozK6AAhMm7_i?usp=sharing) for a Colab using the Gradio app instead!


In [5]:
!python3 -m pip install groq requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 KB 3.7 MB/s eta 0:00:00


In [6]:
import os
from groq import Groq

# Load API key from .env file
api_key = os.getenv("GROQ_API_KEY")

models = [
    'gemma-7b-it', 
    'gemma2-9b-it',
    'mixtral-8x7b-32768',
    'llama3-8b-8192',
    'llama3-70b-8192',
    'llama3-groq-8b-8192-tool-use-preview',
    'llama3-groq-70b-8192-tool-use-preview',
    'llama-3.1-8b-instant',
    'llama-3.1-70b-versatile',
    'llama-guard-3-8b',
]
# Check if API key is set
if not api_key:
    raise ValueError("GROQ_API_KEY is not set in .env file")

# Create a Groq client
groq_client = Groq(api_key=api_key)


# Guia

Uso o groq para:
1. Gerar um 'seed_output' chamando o llm com o 'system_seed.md' como system e uma frase constante.
    - Para gerar ideias de descrição musical

## Esboço 01
```python
with open("system_seed.md", "r") as f:
    system_seed = f.read()

musa_input = "We are the music makers, and we are the dreamers of dreams."

try:
    file_name_responce = groq_client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_seed},
            {"role": "user", "content": musa_input}
        ],
        model=models[8],
        temperature=0.9,  # adjust the temperature to your liking
        max_tokens=1000,  # adjust the max tokens to your liking
        top_p=0.5,  # adjust the top p to your liking
        # stream=True,  # enable streaming
    )
    print(file_name_responce.choices[0].message.content)
except Exception as e:
    print("Error:", str(e))
```

2. Gerar o create_output, usar 'seed_prompt' como input para uma chamada ao llm com o 'system_create.md' como system
    - Para usar o anterir para finalizar uma idea para a descrição musical.

## Esboço 02
```python

# Read system prompt from system.md file
with open("system_create.md", "r") as f:
    system_prompt = f.read()

# Define user input
#user_input = "We are the music makers, and we are the dreamers of dreams."
user_input = file_name_responce.choices[0].message.content

# Create chat completion
try:
    response = groq_client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ],
        model=models[8],
        temperature=0.8,  # adjust the temperature to your liking
        max_tokens=200,  # adjust the max tokens to your liking
        top_p=0.1,  # adjust the top p to your liking
        # stream=True,  # enable streaming
    )
    print(response.choices[0].message.content)
except Exception as e:
    print("Error:", str(e))

```

3. Gerar o 'file_name_output', com o create_output como input e o 'system_generate_file_names.md' como system.
    - Para gerar um nome para os ficheros com base no conteudo produzido no output 2.

## Esboço 03
```python

with open("system_generate_file_names.md", "r") as f:
    system_prompt_titles = f.read()

try:
    file_name_responce = groq_client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt_titles},
            {"role": "user", "content": response.choices[0].message.content}
        ],
        model=models[1],
        temperature=0.2,  # adjust the temperature to your liking
        max_tokens=50,  # adjust the max tokens to your liking
        top_p=0.3,  # adjust the top p to your liking
        # stream=True,  # enable streaming
    )
    print(file_name_responce.choices[0].message.content)
except Exception as e:
    print("Error:", str(e))

```

---

```

```

In [38]:
# CALLING GROQ CHAT COMPLETIONS

from venv import logger
import time

def create_groq_chat_completion(
          system_prompt, 
          user_input, 
          model='llama-3.1-8b-instant', 
          temperature=0.9, 
          max_tokens=1024, 
          top_p=0.8):
    """
    Creates a chat completion.

    Args:
    - system_prompt (str): The system prompt.
    - user_input (str): The user input.
    - model (str): Model name. Defaults to 'llama-3.1-8b-instant'.
    - temperature (float, optional): The temperature. Defaults to 0.9.
    - max_tokens (int, optional): The maximum tokens. Defaults to 1024.
    - top_p (float, optional): The top p. Defaults to 0.8.

    Returns:
    - str: The chat completion response.

    Raises:
    - Exception: If an error occurs.

    This method attempts to create a chat completion using the specified parameters.
    If a 503 Service Unavailable error is encountered, the method will retry the request until it succeeds.

    Error Handling:
    - 503 Service Unavailable: The server is not ready to handle the request, often due to maintenance or overload. The method will wait 2 seconds and retry the request indefinitely until a successful response is received.
    - Other Exceptions: All other exceptions are logged and re-raised to ensure they are not suppressed, allowing for further handling if necessary.
    """
    while True:
        try:
            response = groq_client.chat.completions.create(
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_input}
                ],
                model=model,
                temperature=temperature,
                max_tokens=max_tokens,
                top_p=top_p,
            )
            return response.choices[0].message.content
        
        except Exception as e:
            if "503" in str(e):
                logger.warning("Received 503 Service Unavailable. Retrying...")
                time.sleep(2)  # Wait 2 seconds before retrying
                continue
            else:
                logger.error(f"Error: {str(e)}")
                raise


In [56]:
# CLEAN GENERATED FILENAME AND VERIFIES IF THIS NAME ALREADY EXISTS IN THE OUTPUT FOLDER, IF SO INCREMENTS IT 

import os
import re

def clean_filename(filename):
    """
    Clean the filename by converting it to lowercase, replacing spaces with underscores,
    and removing special characters except underscores and dots.
    
    Args:
    filename (str): The filename to clean.
    
    Returns:
    str: Cleaned filename without any special characters or whitespace.
    """
    # Convert filename to lowercase
    filename = filename.lower()
    
    # Replace whitespace with underscores
    filename = re.sub(r'\s+', '_', filename)
    
    # Remove special characters except underscores and dots
    filename = re.sub(r'[^\w_.]', '', filename)
    
    return filename

def increment_filename(path, filename):
    """
    Returns the filename with the proper increment if a file with the same name already exists in the specified path.
    
    Args:
    path (str): The path to the directory to check in.
    filename (str): The filename to check.
    
    Returns:
    str: The filename with the proper increment.
    """
    # Clean the filename first
    filename = clean_filename(filename)
    
    # Split the filename into name and extension
    name, extension = os.path.splitext(filename)
    
    # Add initial suffix _0
    filename = f"{name}0{extension}"
    filepath = os.path.join(path, filename)
    
    # Initialize the counter starting from 1 since _0 is already added
    counter = 1
    
    # Check if the file exists
    while os.path.isfile(filepath):
        # If the file exists, increment the counter and update the filename
        filename = f"{name}_{counter}{extension}"
        filepath = os.path.join(path, filename)
        counter += 1
    
    return filename


In [10]:
# CREATE AN OUTPUT FOLDER FOR THE ITERATIVE PROCESS

folder_path = "bulk"

try:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created successfully.")
    else:
        print(f"Folder '{folder_path}' already exists.")
except OSError as e:
    print(f"Error creating folder: {e}")

Folder 'bulk' created successfully.


In [59]:
# ROUTINE FOR BULK FILE GENERATION (.wav and .md)

folder_path = "bulk"

with open("system_prompts/system_seed.md", "r") as f:
    system_seed = f.read()

with open("system_prompts/system_create.md", "r") as f:
    system_create = f.read()

with open("system_prompts/system_generate_file_names.md", "r") as f:
    system_prompt_titles = f.read()

seed_input = "We are the music makers, and we are the dreamers of dreams."

import time

def wait_for_completion(result):
    while result is None:
        time.sleep(0.1)  # Wait for 100ms before checking again
    return result

def format_model_output(output, model, temperature, max_tokens, top_p):
    formatted_output = f"""
## Model Output
- **Model:** {model}
- **Temperature:** {temperature}
- **Max Tokens:** {max_tokens}
- **Top P:** {top_p}

{output}
"""
    return formatted_output.strip()

def save_to_markdown(file_path, file_name, *sections):
    content = f"# {file_name}\n\n"
    content += "\n\n---\n\n".join(sections)
    
    with open(f"{file_path}/{file_name}", "w") as f:
        f.write(content)
    
    print(f"Markdown file '{file_name}' has been created.")


while 1:

    seed_output = None
    seed_output = create_groq_chat_completion(system_seed, seed_input, model=models[8], temperature=0.9, max_tokens=1000, top_p=0.7)
    seed_output = wait_for_completion(seed_output)

    time.sleep(1)

    create_output = None
    create_output = create_groq_chat_completion(system_create, seed_output, model=models[8], temperature=0.8, max_tokens=200, top_p=0.5)
    create_output = wait_for_completion(create_output)

    time.sleep(1)

    file_name_output = None
    file_name_output = create_groq_chat_completion(system_prompt_titles, create_output, model=models[1], temperature=0.2, max_tokens=50, top_p=0.3)
    file_name_output = wait_for_completion(file_name_output)

    print(f"This is the current file_name_output:{file_name_output} with leght = {len(file_name_output)}")

    time.sleep(1)

    # After each model call, format a pretty string for viewing in the markdown file:
    seed_formatted = format_model_output(seed_output, models[8], 0.9, 1000, 0.7)
    create_formatted = format_model_output(create_output, models[8], 0.8, 200, 0.5)
    file_name_formatted = format_model_output(file_name_output, models[1], 0.2, 50, 0.3)

    # Fix file name if already exists
    markdown_file_name_output = increment_filename(folder_path, f"{file_name_output}.md")
    wav_file_name_output = increment_filename(folder_path, f"{file_name_output}.wav")

    # Generate audio with the generated description
    res = model.generate([
        create_formatted,
    ],
        progress=True)
    #display_audio(res, 32000)

    # Move the tensor to CPU if it's on GPU and convert to NumPy array
    audio_data = res.cpu().numpy()

    # Save the audio file
    filename = wav_file_name_output
    sample_rate = 32000  # Sample rate used in display_audio
    wavfile.write(f"bulk/{filename}", sample_rate, audio_data)

    # Provide a download link
    #display(FileLink(filename))

    print(seed_output)
    print("---")
    print(create_output)
    print("---")
    print(file_name_output)
    print("---")

    save_to_markdown(folder_path, markdown_file_name_output, seed_formatted, create_formatted, file_name_formatted)

This is the current file_name_output:chamber_jazz_instrumental_piano_violin_trumpet 
 with leght = 48
A fusion of piano arpeggios, soaring violin melodies, and muted trumpet harmonies. Moderate tempo, 6/8 time signature, swaying rhythms. Legato piano, staccato violin, mellow trumpet. Piano, violin, trumpet, double bass, and drums.
---
- **Genre and Tags**: *Genre: Jazz | Subgenres: Chamber Jazz, Jazz Fusion | Tags: Instrumental, Acoustic*
- **Musical Description**: A lilting, wistful piece featuring intricate piano arpeggios in legato style, soaring violin melodies with staccato articulation, and muted trumpet harmonies adding a touch of melancholy. The moderate tempo and 6/8 time signature create a swaying, dance-like feel, while the double bass and drums provide a subtle yet driving rhythmic foundation.
---
chamber_jazz_instrumental_piano_violin_trumpet 

---
Markdown file 'chamber_jazz_instrumental_piano_violin_trumpet_0.md' has been created.
This is the current file_name_output:ser

Received 503 Service Unavailable. Retrying...
Received 503 Service Unavailable. Retrying...
Received 503 Service Unavailable. Retrying...
Received 503 Service Unavailable. Retrying...


This is the current file_name_output:piano_violin_trumpet_melancholy_reflection_chamber_music 
 with leght = 58
Soft piano melody accompanied by legato violin phrases and muted trumpet fills, set against a slow, introspective 4/4 rhythm and minor chord progressions, with subtle woodwind textures.
---
- **Genre and Tags**: *Genre: Classical | Subgenres: Chamber, Instrumental*
- **Musical Description**: A contemplative piano melody unfolds with a simple, yet emotive motif, accompanied by soaring legato violin phrases that add a sense of longing. Muted trumpet fills punctuate the melody, adding a touch of melancholy. The slow, introspective 4/4 rhythm provides a sense of stability, while minor chord progressions evoke a somber mood. Subtle woodwind textures, featuring gentle flute and clarinet whispers, enhance the overall sense of intimacy and vulnerability, creating a poignant and reflective atmosphere.
---
piano_violin_trumpet_melancholy_reflection_chamber_music 

---
Markdown file 'pi

KeyboardInterrupt: 

Ran for 42 mins